**MetalPy** + SAP2000
-----------------

Este notebook apresenta um exemplo de aplicação do pacote **MetalPy**, integrando-o ao software de análise estrutural SAP2000.

Neste exemplo **MetalPy** é utilizado para fazer a verificação das barras de um modelo de treliça apresentado na figura 1. A treliça é formada por 33 barras, todas elas constítuidas de perfis tubo retangulares laminados e está submetido a um carregamento de aproximadamente 50kN em cada um de seus nós.

<h1 align="center">
<img src="img/modelo_sap_carregamento.png">
</h1><br>

A treliça apresentada é formado por 4 diferentes tipos de perfis como mostrado na figura 2, todos eles de aço ASTM A992:

<h1 align="center">
<img src="img/modelo_sap_secoes.png">
</h1><br>

 * Barras do banzo superior - perfil TQ160X160X8
 * Barras do banzo inferior - perfil TQ100X80X8.8
 * Barras dos motantes - perfil TR150X120X6.4
 * Barras das diagonais -  perfil TQ100X80X8.8

A obtenção dos esforços solicitantes é realizada através da análise estrutural realizado no SAP2000 e a verificação dos perfis é realizada com as funções do pacote **Metalpy** com os critérios de verificação da NBR8800:2008.

In [1]:
#Importação do scrip de inicialização do SAP2000
from iniciar_sap import IniciarSAP

#Importando as classes necessárias do pacote Metalpy
from metalpy.perfis import TuboRet
from metalpy.material import Aco
from metalpy.normas import NBR8800

In [2]:
# Iniciando o SAP2000
modelo_sap = IniciarSAP()
modelo_sap.InitializeNewModel()

#Abrindo o Modelo de Analise no Sap
FileName = 'modelo_sap\TrelicaMPySap.sdb' #Conferir o Diretório onde o arquivo do Modelo está Salvo
ret = modelo_sap.File.OpenFile(FileName)

In [3]:
# Verificando o se o modelo está com a análise processada
if not modelo_sap.GetModelIsLocked():
    
    # Rodando a análise caso ela não esteja processada
    modelo_sap.Analyze.RunAnalysis()

In [12]:
#Obtendo o nome das barras da treliça do SAP2000
barras_sap = modelo_sap.FrameObj.GetNameList()[1]

nome_barras = list()
status = list()
aproveitamento = list()

for barra in barras_sap:

    # -----------------------------
    # OBTENÇÃO DOS DADOS DO SAP2000
    # -----------------------------

    # Obtendo o nome do perfil da barra
    perfil = modelo_sap.FrameObj.GetSection(barra)[0]

    # Obtendo as propriedades do material da barra
    mat_perfil_sap = modelo_sap.PropFrame.GetMaterial(perfil)[0]

    E, v = modelo_sap.PropMaterial.GetMPIsotropic(mat_perfil_sap)[:2]
    fy, fu = modelo_sap.PropMaterial.GetOSteel_1(mat_perfil_sap)[:2]

    # Obtendo o comprimento a as solicitções das barras
    modelo_sap.Results.Setup.SetCaseSelectedForOutput('1')
    resultados = modelo_sap.Results.FrameForce(barra, 0)

    klx = kly = klz = resultados[2][-1] # Comprimento da barra
    Nsd = max(resultados[8]) # Esforço axial na barra

    # -------------------------------------
    # REALIZANDO A VERIFICAÇÃO COM METALPY
    # -------------------------------------

    aco = Aco(E, v, fy, fu, 0.6) # Criando o aço correspondente 
    perfil_mp = TuboRet(perfil, aco, und='cm') # Criando o perfil correspondente

    if Nsd > 0:
        Nrd = NBR8800.Ntrd_brt(perfil_mp)
        
    elif Nsd <= 0:
        Nrd = NBR8800.Ncrd(perfil_mp, klx, kly, klz)

    

    if Nrd >= abs(Nsd):
        nome_barras.append('Barra ' + barra)
        status.append('Ok!')
        aproveitamento.append(str(round(abs(Nsd / Nrd)*100, 2)) +  ' %')


    else:
        nome_barras.append(barra)
        status.append('** NÃO PASSOU NA VERIFICAÇÃO**')
        aproveitamento.append(str(round(abs(Nsd / Nrd)*100, 2) +  ' %'))

Resultados
----------

In [11]:
import pandas as pd

columns = 'Status Nsd/Nrd'.split()
index = nome_barras
data = zip(status, aproveitamento)

df = pd.DataFrame(data,index=index,columns=columns)
df

,Status,Nsd/Nrd
Barra 1,Ok!,92.31 %
Barra 2,Ok!,69.28 %
Barra 3,Ok!,46.2 %
Barra 4,Ok!,23.11 %
Barra 5,Ok!,11.14 %
Barra 6,Ok!,23.11 %
Barra 7,Ok!,46.2 %
Barra 8,Ok!,69.28 %
Barra 9,Ok!,92.31 %
Barra 10,Ok!,0.0 %
